# Feature documentation

## Features Actuales

   ###  MaxMin Date: 
 - Saco la date maxima y minima por usuario y de ahi obtengo features de esas dates
    
## Features probados que no funcionaron y otras cosas que tampoco:

   ### Cantidad de eventos por tipo
  - Numero de veces que realizo cada evento en dataframe
   ### Algoritmo 0.63 con RandomFores
  - Use random forest para ver la importancia de lo features de las busquedas y de los modelos y me quede con los mejores 10, el resultado fue menor con 0.61
   
## Best Score: 0.71 
  - commit 0.71 
  - Max, min date.
  - 
               
## Cosas a probar:
  - features 
  - model countvector (sin random forest seleccionando un top)
  - most viewed and searched
  - lead and conversion
  - most common search (sin random forest seleccionando un top)
 
 
 
## Acotaciones
     

In [6]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt
from scipy.sparse.csr import csr_matrix
from sklearn.model_selection import train_test_split
import featuretools as ft

pd.options.mode.chained_assignment = None


In [2]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [3]:
df['session_id'] = df.index

In [4]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,session_id
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [5]:
df_person = df['person'].drop_duplicates().to_frame()

In [6]:
df['session']=df.index

In [7]:
es = ft.EntitySet(id = 'person')
es = es.entity_from_dataframe(entity_id = 'person_id', dataframe = df_person, index = 'person')

In [8]:

df_session = df

In [9]:
es = es.entity_from_dataframe(entity_id = 'session', dataframe = df_session, 
                            index = 'session_id')


In [10]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 1]
    session [Rows: 2341681, Columns: 25]
  Relationships:
    No relationships

In [11]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['person_id']['person'],
                                    es['session']['person'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [12]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 1]
    session [Rows: 2341681, Columns: 25]
  Relationships:
    session.person -> person_id.person

In [5]:

%time features, feature_names = ft.dfs(entityset = es, target_entity = 'person_id', agg_primitives = ['mean', 'max', 'percent_true', 'last'],trans_primitives = ['years', 'month', 'subtract', 'divide'])

NameError: name 'ft' is not defined

In [14]:
features, feature_names = ft.encode_features(features, feature_names)
features.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),...,MODE(session.MONTH(timestamp)) = 1,MODE(session.MONTH(timestamp)) is unknown,MODE(session.WEEKDAY(timestamp)) = 2,MODE(session.WEEKDAY(timestamp)) = 1,MODE(session.WEEKDAY(timestamp)) = 3,MODE(session.WEEKDAY(timestamp)) = 0,MODE(session.WEEKDAY(timestamp)) = 4,MODE(session.WEEKDAY(timestamp)) = 5,MODE(session.WEEKDAY(timestamp)) = 6,MODE(session.WEEKDAY(timestamp)) is unknown
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,...,0,0,0,0,1,0,0,0,0,0
00091926,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,...,0,0,0,0,0,0,0,0,1,0
00091a7a,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,...,0,0,0,0,0,1,0,0,0,0
000ba417,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,...,0,0,0,0,1,0,0,0,0,0
000c79fe,39776.0,4924515,0.000000,647464.281020,9944.0,2174099,0.000000,2.654337,9944.0,57876,...,0,0,0,1,0,0,0,0,0,0


In [15]:
features_automaticos = features

In [16]:
features_automaticos.head(1)

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),...,MODE(session.MONTH(timestamp)) = 1,MODE(session.MONTH(timestamp)) is unknown,MODE(session.WEEKDAY(timestamp)) = 2,MODE(session.WEEKDAY(timestamp)) = 1,MODE(session.WEEKDAY(timestamp)) = 3,MODE(session.WEEKDAY(timestamp)) = 0,MODE(session.WEEKDAY(timestamp)) = 4,MODE(session.WEEKDAY(timestamp)) = 5,MODE(session.WEEKDAY(timestamp)) = 6,MODE(session.WEEKDAY(timestamp)) is unknown
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.1301,3372.0,1505383,...,0,0,0,0,1,0,0,0,0,0


In [17]:
features_automaticos, feature_names = ft.encode_features(features_automaticos, feature_names)
features_automaticos.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),...,MODE(session.MONTH(timestamp)) = 1,MODE(session.MONTH(timestamp)) is unknown,MODE(session.WEEKDAY(timestamp)) = 2,MODE(session.WEEKDAY(timestamp)) = 1,MODE(session.WEEKDAY(timestamp)) = 3,MODE(session.WEEKDAY(timestamp)) = 0,MODE(session.WEEKDAY(timestamp)) = 4,MODE(session.WEEKDAY(timestamp)) = 5,MODE(session.WEEKDAY(timestamp)) = 6,MODE(session.WEEKDAY(timestamp)) is unknown
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,...,0,0,0,0,1,0,0,0,0,0
00091926,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,...,0,0,0,0,0,0,0,0,1,0
00091a7a,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,...,0,0,0,0,0,1,0,0,0,0
000ba417,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,...,0,0,0,0,1,0,0,0,0,0
000c79fe,39776.0,4924515,0.000000,647464.281020,9944.0,2174099,0.000000,2.654337,9944.0,57876,...,0,0,0,1,0,0,0,0,0,0


In [18]:
features_automaticos['person'] = features_automaticos.index
features_automaticos = features_automaticos.reset_index(drop=True)
features_automaticos.head(3)

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),...,MODE(session.MONTH(timestamp)) is unknown,MODE(session.WEEKDAY(timestamp)) = 2,MODE(session.WEEKDAY(timestamp)) = 1,MODE(session.WEEKDAY(timestamp)) = 3,MODE(session.WEEKDAY(timestamp)) = 0,MODE(session.WEEKDAY(timestamp)) = 4,MODE(session.WEEKDAY(timestamp)) = 5,MODE(session.WEEKDAY(timestamp)) = 6,MODE(session.WEEKDAY(timestamp)) is unknown,person
0,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,...,0,0,0,1,0,0,0,0,0,0008ed71
1,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,...,0,0,0,0,0,0,0,1,0,00091926
2,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,...,0,0,0,0,1,0,0,0,0,00091a7a


## XGboost entrenamiento

In [44]:
#df_train = df_labels.merge(features_automaticos , left_on='person', right_on='person' , how='left')

df_train = df_labels.merge(selected_features , left_on='person', right_on='person' , how='left')


Los labels me dan mi set para entrenar, los que no se encuentran en labels tengo que predecirlos

In [45]:
df_train.head()

,person,label,COUNT(session),MAX(session.CHARACTERS(skus)),MAX(session.sku),MEAN(session.CHARACTERS(skus)),MEAN(session.sku),MIN(session.sku),MODE(session.DAY(timestamp)) = 18,MODE(session.DAY(timestamp)) = 21,...,NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.url),SKEW(session.CHARACTERS(skus)),SKEW(session.sku),STD(session.CHARACTERS(skus)),STD(session.sku),SUM(session.CHARACTERS(skus)),SUM(session.NUMWORDS(skus)),SUM(session.sku)
0,0566e9c1,0,68,62,10519.0,17.970588,4629.480000,87.0,0,0,...,1,3,1,0.925216,0.725500,28.051110,3059.600084,1222,68,115737.0
1,6ec7ee77,0,2,0,NaN,0.000000,NaN,NaN,0,0,...,0,0,1,NaN,NaN,0.000000,NaN,0,2,0.0
2,abe7a2fb,0,96,68,9412.0,17.250000,3434.125000,1301.0,0,0,...,0,4,6,0.974114,1.602912,26.682145,2383.991959,1656,96,109892.0
3,34728364,0,37,63,13277.0,12.000000,8560.291667,3107.0,0,0,...,0,3,0,1.603400,-0.382143,24.063573,3123.159931,444,37,205447.0
4,87ed62de,0,17,0,6413.0,0.000000,6408.272727,6385.0,1,0,...,0,1,1,0.000000,-2.403223,0.000000,8.684364,0,17,70491.0


Si ven aca, de la columna label en adelante tenemos los features.

In [46]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X = X.fillna(0)
X.head()

,COUNT(session),MAX(session.CHARACTERS(skus)),MAX(session.sku),MEAN(session.CHARACTERS(skus)),MEAN(session.sku),MIN(session.sku),MODE(session.DAY(timestamp)) = 18,MODE(session.DAY(timestamp)) = 21,MODE(session.DAY(timestamp)) = 22,MODE(session.DAY(timestamp)) = 23,...,NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.url),SKEW(session.CHARACTERS(skus)),SKEW(session.sku),STD(session.CHARACTERS(skus)),STD(session.sku),SUM(session.CHARACTERS(skus)),SUM(session.NUMWORDS(skus)),SUM(session.sku)
0,68,62,10519.0,17.970588,4629.480000,87.0,0,0,1,0,...,1,3,1,0.925216,0.725500,28.051110,3059.600084,1222,68,115737.0
1,2,0,0.0,0.000000,0.000000,0.0,0,0,0,0,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0,2,0.0
2,96,68,9412.0,17.250000,3434.125000,1301.0,0,0,0,0,...,0,4,6,0.974114,1.602912,26.682145,2383.991959,1656,96,109892.0
3,37,63,13277.0,12.000000,8560.291667,3107.0,0,0,0,0,...,0,3,0,1.603400,-0.382143,24.063573,3123.159931,444,37,205447.0
4,17,0,6413.0,0.000000,6408.272727,6385.0,1,0,0,0,...,0,1,1,0.000000,-2.403223,0.000000,8.684364,0,17,70491.0


Separamos los datos para hacer xgboost de la siguiente forma


|Variable |Contiene|
|------------------------|-----------------------------------------------------|
|X| features que usa xgboost son solo numeros es decir que sacamos a la persona   |
|y| label de cada persona|

## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [47]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

Este es el arbol con sus hiperparametros

In [48]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

Aca vuelvo a separar los datos para poder realizar una metrica interna y ver masomenos como performan las cosas que hago

In [49]:
persons = df_labels['person']
persons_to_predict = (features_automaticos.loc[~features_automaticos['person'].isin(persons)])['person'].to_frame()
persons_to_predict.head()

,person
1,00091926
2,00091a7a
3,000ba417
5,000e4d9e
6,000e619d


Obtengo las personas a predecir!

In [50]:
X_predict = features_automaticos.loc[features_automaticos['person'].isin(persons_to_predict['person'])]
#X_predict = selected_features.loc[selected_features['person'].isin(persons_to_predict['person'])]


X_predict = X_predict.drop(['person'], axis=1)
X_predict.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),...,MODE(session.MONTH(timestamp)) = 1,MODE(session.MONTH(timestamp)) is unknown,MODE(session.WEEKDAY(timestamp)) = 2,MODE(session.WEEKDAY(timestamp)) = 1,MODE(session.WEEKDAY(timestamp)) = 3,MODE(session.WEEKDAY(timestamp)) = 0,MODE(session.WEEKDAY(timestamp)) = 4,MODE(session.WEEKDAY(timestamp)) = 5,MODE(session.WEEKDAY(timestamp)) = 6,MODE(session.WEEKDAY(timestamp)) is unknown
1,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,...,0,0,0,0,0,0,0,0,1,0
2,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,...,0,0,0,0,0,1,0,0,0,0
3,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,...,0,0,0,0,1,0,0,0,0,0
5,1469071.0,128149995,3009.507486,584744.700882,15730.0,2173028,1.341854,2.423675,325.0,2034,...,0,0,1,0,0,0,0,0,0,0
6,200221.0,83902787,4482.725280,594587.095191,13404.0,2259744,-0.267465,0.656392,129.0,737843,...,0,0,0,0,1,0,0,0,0,0


Necesito tener el mismo dataframe que tenia cuando lo entrene pero ahora para predecir, en este caso <b>X</b>

In [51]:
my_classifier1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

Entreno al arbol!

In [52]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.7319731923559075

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

In [53]:
entrie = my_classifier1.predict_proba(X_predict)[:,1]

ValueError: feature_names mismatch: ['COUNT(session)', 'MAX(session.CHARACTERS(skus))', 'MAX(session.sku)', 'MEAN(session.CHARACTERS(skus))', 'MEAN(session.sku)', 'MIN(session.sku)', 'MODE(session.DAY(timestamp)) = 18', 'MODE(session.DAY(timestamp)) = 21', 'MODE(session.DAY(timestamp)) = 22', 'MODE(session.DAY(timestamp)) = 23', 'MODE(session.DAY(timestamp)) = 29', 'MODE(session.DAY(timestamp)) = 30', 'MODE(session.DAY(timestamp)) = 31', 'MODE(session.DAY(timestamp)) is unknown', 'MODE(session.MONTH(timestamp)) = 5', 'MODE(session.WEEKDAY(timestamp)) = 1', 'MODE(session.WEEKDAY(timestamp)) = 2', 'MODE(session.WEEKDAY(timestamp)) = 3', 'MODE(session.WEEKDAY(timestamp)) = 4', 'MODE(session.WEEKDAY(timestamp)) = 5', 'MODE(session.WEEKDAY(timestamp)) = 6', 'MODE(session.browser_version) = Chrome 66.0', 'MODE(session.browser_version) is unknown', 'MODE(session.campaign_source) = criteo', 'MODE(session.campaign_source) = google', 'MODE(session.campaign_source) = rtbhouse', 'MODE(session.camp

Valores a subir a kaggle, lo que sigue son transformaciones a dataframe para convertirlo en csv, y algunas cosas para darme idea tipo por donde andan los valores normales en los resultados que me da

In [54]:
entrie

array([0.20607466, 0.34204352, 0.20792755, ..., 0.25726742, 0.3400402 ,
       0.18092427], dtype=float32)

In [55]:
seriesita = pd.Series(entrie)

In [56]:
df_entrie = persons_to_predict
df_entrie['label'] = seriesita

In [57]:
df_entrie.head()

,person,label
1,00091926,0.342044
2,00091a7a,0.207928
3,000ba417,0.207928
5,000e4d9e,0.188744
6,000e619d,0.195176


In [58]:
df_entrie = df_entrie.fillna(0)

In [59]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

In [60]:
df_entrie.to_csv(path_or_buf = 'lastautomated.csv', index = False)

In [61]:
df_entrie.shape

(19415, 2)

In [62]:
df_entrie['label'].nlargest()

10587    0.551074
17863    0.550978
7425     0.536125
10359    0.521153
10627    0.510242
Name: label, dtype: float32

 # Random Forest feature importance
   - Algoritmo usado para sacar importancia de los features y ver cuales no nos estan sirviendo al modelo
 #### Es parecido a lo que hice en xgboost con algunas cosas magicas, que use para ponerlo en df y ver resultados
 
  ## NOTAR :
   - Que use la X de xgboost
   - Y que el codigo abajo del dataframe comentado puede funcionar para separar los df pidiendo que nos deje las columnas esas

In [63]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
zipped = zip(map(lambda x: round(x, 4), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

/home/amaherok/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [64]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance.sort_values('importance', ascending=False)

,importance,feature
92,0.0508,SKEW(session.sku)
4,0.0444,MEAN(session.sku)
97,0.0433,SUM(session.sku)
93,0.0428,STD(session.CHARACTERS(skus))
2,0.0416,MAX(session.sku)
94,0.0411,STD(session.sku)
95,0.0382,SUM(session.CHARACTERS(skus))
5,0.0378,MIN(session.sku)
3,0.0354,MEAN(session.CHARACTERS(skus))
91,0.0309,SKEW(session.CHARACTERS(skus))


In [66]:
feats_ausar =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(100))]
feats_servibles = feats_ausar['feature'].tolist()
feats_servibles.append('person')
feats_servibles =[ x for x in feats_servibles if "sku" not in x ]


selected_features = features_automaticos[feats_servibles]
selected_features.head()

,COUNT(session),MODE(session.DAY(timestamp)) = 18,MODE(session.DAY(timestamp)) = 21,MODE(session.DAY(timestamp)) = 22,MODE(session.DAY(timestamp)) = 23,MODE(session.DAY(timestamp)) = 29,MODE(session.DAY(timestamp)) = 30,MODE(session.DAY(timestamp)) = 31,MODE(session.DAY(timestamp)) is unknown,MODE(session.MONTH(timestamp)) = 5,...,NUM_UNIQUE(session.event),NUM_UNIQUE(session.model),NUM_UNIQUE(session.operating_system_version),NUM_UNIQUE(session.region),NUM_UNIQUE(session.screen_resolution),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.url),person
0,6,0,0,0,0,0,0,0,0,1,...,3,3,1,1,1,0,0,2,0,0008ed71
1,448,0,0,0,0,0,0,0,1,1,...,5,36,1,1,1,0,0,5,11,00091926
2,10,0,0,0,0,0,0,0,1,0,...,4,3,1,1,1,0,0,3,1,00091a7a
3,206,0,0,0,0,0,0,0,0,1,...,8,26,1,1,1,0,0,6,1,000ba417
4,17,0,0,0,0,1,0,0,0,1,...,7,1,1,1,1,7,0,1,1,000c79fe
